# Triplet Train

使用ipynb笔记本来对python代码进行逐步调试处理/训练

首先，读取数据。

In [1]:
import logging
import torch
import numpy as np
import tritrain
import dataset
import torch.optim as optim

from dissimilarity import calculate_cs_dissimilarity_matrix
from dataset import read_all_of_huaweicup,read_npy_of_file,npy_file_name_converter

from geodesic import construct_knn_graph,compute_shortest_paths_dijkstra,compute_shortest_paths_dijkstra_cugraph

# 尝试导入 cugraph 和 cudf，如果失败则使用 networkx
try:
    import cudf
    import cugraph
    GPU_AVAILABLE = True
except ImportError:
    GPU_AVAILABLE = False

## 2. 数据读入及预处理
使用 huawei 杯官方给的示例代码，导入数据。

准备数据集，首先定义有关函数。

需要注意，我们的数据集每一个对应着不同的扇区，这里先导一个，每个文件导出的时候也对应不同扇区，训练三个模型。

In [2]:
print("<<< Welcome to 2024 Wireless Algorithm Contest! >>>\n")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Round_idx = 1
File_idx = 1
bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num, anch_pos, H, d_geo = read_all_of_huaweicup(Round_idx,File_idx)

# 初始化日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


<<< Welcome to 2024 Wireless Algorithm Contest! >>>

Processing Round 1 Case 1
Loading configuration data file
Loading input position file
Loading input CSI data of Case 1
Loading Channel CSI succeed
Loading GEO data succeed
[[   0.         2339.73608398 2482.65429688 ... 2290.90087891
  3095.56591797 1301.00170898]
 [2339.73608398    0.         1570.17565918 ... 1972.58605957
  1890.33178711 2245.20751953]
 [2482.65429688 1570.17565918    0.         ... 1390.14709473
  2452.04980469 2113.81030273]
 [2267.93457031  932.56329346 1780.73571777 ... 2248.97143555
  1986.77770996 2143.52075195]
 [3154.51660156 2009.21484375 2895.19995117 ... 2991.2109375
  1568.25439453 3070.37939453]]
(20000, 20000)


## 3.网络训练

Triplet Neural Networks

![Triplet Neural Networks](/home/xmax/Desktop/huaweicup/pic/img.png)

### 3.1 Triplet Selection

输入为Dgeo矩阵，为两个采样点之间的接地线距离，输出为 f(i) , f(j) , f(k) ，其中 i 为2000个锚点， j 为反例，距离 i 很远的例子， k 为正例距离 i 很近的例子。
有一个参数Q，用来衡量近的点有多近，远的点有多远，是一个可调参数。
论文中，Q=0.02-0.2，此处为可调参数。

### 3.2 Embedded 网络


### 3.3 Triplet Loss


In [3]:
# 创建模型
embed_net = tritrain.TripletNet().to(device)

# 初始化损失函数，给入参数 margin
margin = 1.0
criterion = tritrain.TripletLoss(margin).to(device)

H_real = H.real
H_imag = H.imag
H_combined = np.stack((H_real, H_imag), axis=2)
H_combined = H_combined.reshape(tol_samp_num, 4, ant_num, sc_num)

optimizer = optim.Adam(embed_net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# 训练网络
tritrain.train_triplet_network(embed_net, criterion, optimizer, scheduler , H_combined, d_geo, init_q=0.1)

# 保存网络实例
torch_net_file = dataset.tensor_file_name_converter(1,1)
torch.save(embed_net.state_dict(), torch_net_file)
print("Model saved as "+ torch_net_file)


使用cuda进行计算


Epoch 1/20:  16%|█▋        | 103/625 [00:07<00:32, 15.89it/s]

Epoch [1/20], Batch [100], Q: 0.3000, Loss: 18.1153


Epoch 1/20:  32%|███▏      | 203/625 [00:13<00:26, 15.98it/s]

Epoch [1/20], Batch [200], Q: 0.3000, Loss: 0.7521


...

Epoch [19/20], Batch [100], Q: 0.0480, Loss: 0.0778


Epoch [19/20] Completed, Learning Rate: 0.000100


Epoch 20/20:  64%|██████▎   | 100/157 [00:32<00:17,  3.35it/s]

Epoch [20/20], Batch [100], Q: 0.0340, Loss: 0.0758


Epoch [20/20] Completed, Learning Rate: 0.000010
Model saved as ./CompetitionData1/Round1NET11721400377.7985215.pth
